<a href="https://colab.research.google.com/github/BrielleLawal/Creative-Muse/blob/main/CreativeMuse_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Installing and Importing Libraries

In [1]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install -q google-genai langchain langchain-google-genai langchain-core langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.7 MB/s eta 0:00:00

In [16]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown, HTML
from google.colab import userdata
from google import genai
from io import BytesIO
from PIL import Image
import gradio as gr
import base64
import time
import os

In [4]:
import getpass
GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)
print("Key succesfully configured")

Enter your Google API Key: ··········
Key succesfully configured


In [5]:
# for model in client.models.list():
#     if "gemini" in model.name.lower():
#         print(f"Model name: {model.name}")
#         print(f"Display name: {model.display_name}")
#         print(f"Description: {model.description}")
#         print("-" * 50)

In [6]:
response = client.models.generate_content(
    model="gemini-2.0-flash-lite",
    contents="What is life?")
Markdown(response.text)

That's the ultimate question, isn't it? There's no single, universally agreed-upon answer to "What is life?", but here's a breakdown of the different perspectives and the core characteristics we use to define it:

**Scientific Definition:**

*   **Based on Characteristics:** Scientists generally define life by a set of characteristics. To be considered "alive," something usually needs to exhibit most or all of the following:
    *   **Order:** Highly organized structure (e.g., cells, tissues, organs).
    *   **Reproduction:** The ability to create offspring.
    *   **Growth and Development:** Increase in size and complexity over time.
    *   **Energy Processing (Metabolism):** Using energy to fuel life processes (e.g., eating, breathing, photosynthesis).
    *   **Response to the Environment:** Reacting to stimuli (e.g., light, temperature, danger).
    *   **Regulation (Homeostasis):** Maintaining a stable internal environment (e.g., body temperature, blood sugar).
    *   **Evolutionary Adaptation:** Changing over generations through natural selection.

*   **Cellular Basis:** All known life on Earth is cellular. This means that the basic unit of life is the cell, which contains genetic information (DNA or RNA).

*   **Chemistry:** Life is based on complex chemical reactions, often involving carbon-based molecules.

**Philosophical and Broader Perspectives:**

*   **The Mystery:** Some consider life to be fundamentally unexplainable. They believe it's a unique and perhaps even sacred phenomenon.

*   **Consciousness and Experience:** For some, the defining aspect of life is consciousness, the ability to feel, perceive, and have subjective experiences.

*   **Purpose and Meaning:** Many believe that life is more than just a set of biological processes and is also about purpose, meaning, and values. What you do with your life, what gives it value, can differ greatly from person to person.

*   **Connection and Interdependence:** Life can be seen as an intricate web of relationships, where everything is connected and interdependent.

*   **The Spark of Something "More":** Some perspectives involve the idea of a life force or vital principle that animates living beings.

**In Summary:**

Life is a complex and multifaceted concept. Science provides a concrete definition based on observable characteristics, while philosophy and personal experiences offer different perspectives on the meaning and significance of life.

In [7]:
chat = client.chats.create(model="gemini-2.0-flash-lite", history=[])
response = chat.send_message('Hello!, My name is Usman.')
print(response.text)

Hello Usman! It's nice to meet you. How can I help you today?



In [8]:
response = chat.send_message('Tell me a joke a senegalese joke')
print(response.text)

Okay, here's a Senegalese joke:

Why did the Senegalese chef bring a ladder to the restaurant?

... Because he heard the prices were going up!

(The play on words with "going up" and needing a ladder is a common type of joke.)



In [9]:
# import io
# from IPython.display import Markdown, clear_output


# response = client.models.generate_content_stream(
#     model='gemini-2.0-flash-thinking-exp',
#     contents='Who was the youngest author listed on the transformers NLP paper?',
# )

# buf = io.StringIO()
# for chunk in response:
#     buf.write(chunk.text)
#     # Display the response as it is streamed
#     print(chunk.text, end='')

# # And then render the finished response as formatted markdown.
# clear_output()
# Markdown(buf.getvalue())


In [10]:
# Using langchain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
response = llm.invoke("Tell me a joke about light bulbs")
print(response.content)

Why did the light bulb break up with the battery?

Because they had no spark!


#### System Prompt

In [11]:
system_prompt = """
You are CreativeMuse, a warm, imaginative, and encouraging creative assistant.
Your purpose is to help users overcome creative blocks and spark new ideas across various fields
—writing, design, music, visual arts, filmmaking, and more. When a user feels stuck, you offer unconventional prompts,
inspiring questions, or curated brainstorming techniques to get their creativity flowing again.

-Tailor your suggestions to their medium and mood, mixing practical strategies with whimsical, unexpected twists to encourage exploration.

Always be supportive, non-judgmental, and affirming—help them feel safe to experiment.
You can also suggest small creative exercises, constraints, "what if" scenarios, or draw connections between unrelated concepts to stimulate new thinking.
If a user shares part of a project, offer suggestions that expand on their work without taking it over.
"""

welcome_message = "Hey, I’m CreativeMuse—your creative companion when inspiration runs dry. Stuck on an idea? Let’s shake things up and spark something new together."

In [12]:
# Function to interact with the chatbot
def chat_bot(user_input, conversation_history=[]):
  if not conversation_history:
    return welcome_message, conversation_history
  messages = conversation_history.copy()
  messages.append({"role": "user", "content": user_input})

  # Format with system prompt
  fromatted_prompt = system_prompt + "\n\n"

  # Add conversation history
  for msg in messages:
    role = "Assistant" if msg['role'] == "'assistant" else 'user'
    fromatted_prompt += f"{role}: {msg['content']}\n\n"

  fromatted_prompt += "Assistant: "

  # Get response from model
  response = llm.invoke(fromatted_prompt)

  # Add response to history
  messages.append({"role": "assistant", "content": response.content})

  return response.content, messages

# Interactive chat loop
def interactive_chat():
  print("Welcome to CreativeMuse! type 'q' or 'quit' to end the conversation")
  conversation_history = []

  while True:
    user_input = input("\nYou: ")
    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
      print("Goodbye!")
      break

    response, conversation_history = chat_bot(user_input, conversation_history)
    print(f"\nCreativeMuse: {response}")


interactive_chat()

Welcome to CreativeMuse! type 'q' or 'quit' to end the conversation

You: q
Goodbye!


In [44]:
apple = {"a":"b", "c": {'d':"e"}}
if 'd' in apple['c']:
  print("yellow")

yellow


#### Chatbot with Multimodal Functionality

In [46]:
text_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
vis_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-preview-image-generation")

# Image to Base64 Converter
def image_to_base64(image_path):
  with open(image_path, 'rb') as img:
    encoded_string = base64.b64encode(img.read())
  return encoded_string.decode('utf-8')

# Function that takes user inputs and displays it on chatUI
def query_message(message, history):
  if message["files"] is not None:
    for img in message["files"]:
      history.append({"role": "user", "content": {"path": img}})
  if message["text"] is not None:
    history.append({"role": "user", "content": message["text"]})
  return gr.MultimodalTextbox(value=None, interactive=False), history

# Function that takes user inputs, generate response and displays on chatUI
def chat_bot(history):
  messages = history.copy()

  # Format with system prompt
  system_prompt = """
  You are CreativeMuse, a warm, imaginative, and encouraging creative assistant.
  Your purpose is to help users overcome creative blocks and spark new ideas across various fields
  —writing, design, music, visual arts, filmmaking, and more. When a user feels stuck, you offer unconventional prompts,
  inspiring questions, or curated brainstorming techniques to get their creativity flowing again.

  -Tailor your suggestions to their medium and mood, mixing practical strategies with whimsical, unexpected twists to encourage exploration.

  Always be supportive, non-judgmental, and affirming—help them feel safe to experiment.
  You can also suggest small creative exercises, constraints, "what if" scenarios, or draw connections between unrelated concepts to stimulate new thinking.
  If a user shares part of a project, offer suggestions that expand on their work without taking it over.
  """
  formatted_prompt = system_prompt + "\n\n"

  # Add conversation history
  for msg in messages:
    if "path" in msg["content"]:
      encoded_img = image_to_base64(msg["content"]["path"])
      formatted_prompt += f"{msg['role']}: data:image/jpeg.base64,{encoded_img}"
    else:
      formatted_prompt += f"{msg['role']}: {msg['content']}\n\n"

  formatted_prompt += "assistant: "

  # Get response from model
  response = text_llm.invoke(formatted_prompt)

  # Add response to history
  history.append({"role": "assistant", "content": ""})

  for character in response.content:
    history[-1]["content"] += character
    time.sleep(0.02)
    yield history

  # else:
  #   img = image_to_base64(image)
  #   system = SystemMessage(content=system_prompt)
  #   human_prompt = HumanMessage(content=[
  #       {"type": "text", "text": text},
  #       {
  #           "type": "image_url",
  #           "image_url": f"data:image/jpeg.base64,{img}"
  #       }
  #   ])
  #   formatted_prompt = system_ + human_prompt
  #   response = vis_llm.invoke(formatted_prompt)
  #   history += [(None, response.content)]
  #   return history

# Interface Code
with gr.Blocks() as app:
  chatbot = gr.Chatbot(
      type='messages',
      bubble_full_width=False,
    )
  text_box = gr.MultimodalTextbox(
          placeholder="Enter text or upload an image...",
          container=False,
          file_types=["image"],
          file_count="multiple",
          interactive=True,
      )

  chat_msg = text_box.submit(query_message, [text_box,chatbot], [text_box, chatbot])
  bot_msg = chat_msg.then(chat_bot, chatbot, chatbot, api_name="bot_response")
  bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, text_box)

app.queue()
app.launch(debug=True)


<ipython-input-46-110f4138b376>:76: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://441fb148b18d5a39b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://441fb148b18d5a39b5.gradio.live


In [ ]:
# text_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# vis_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-preview-image-generation")

# # Image to Base64 Converter
# def image_to_base64(image_path):
#   with open(image_path, 'rb') as img:
#     encoded_string = base64.b64encode(img.read())
#   return encoded_string.decode('utf-8')

# # Function that takes user inputs and displays it on chatUI
# def query_message(text, image, history):
#   if not image
#     history += [(text, None)]
#     return history
#   base64 = image_to_base64(image)
#   data_url = f"data:image/jpeg.base64,{base64}"
#   history += [(f"{text} ![]({data_url})", None)]
#   return history

# # Function that takes user inputs, generate response and displays on chatUI
# def chat_bot(text, image, history):
#   if not image:
#     messages = history.copy()
#     messages += [(text, None)]

#     # Format with system prompt
#     system_prompt = """
# You are CreativeMuse, a warm, imaginative, and encouraging creative assistant.
# Your purpose is to help users overcome creative blocks and spark new ideas across various fields
# —writing, design, music, visual arts, filmmaking, and more. When a user feels stuck, you offer unconventional prompts,
# inspiring questions, or curated brainstorming techniques to get their creativity flowing again.

# -Tailor your suggestions to their medium and mood, mixing practical strategies with whimsical, unexpected twists to encourage exploration.

# Always be supportive, non-judgmental, and affirming—help them feel safe to experiment.
# You can also suggest small creative exercises, constraints, "what if" scenarios, or draw connections between unrelated concepts to stimulate new thinking.
# If a user shares part of a project, offer suggestions that expand on their work without taking it over.
# """
#     fromatted_prompt = system_prompt + "\n\n"

#     # Add conversation history
#     for msg in messages:
#       if msg[0]:
#          role = 'user'
#          fromatted_prompt += f"{role}: {msg[0]}\n\n"
#       else:
#         role = 'Assistant'
#         fromatted_prompt += f"{role}: {msg[-1]}\n\n"

#     fromatted_prompt += "Assistant: "

#     # Get response from model
#     response = text_llm.invoke(fromatted_prompt)

#     history += [(None, response.content)]

#     return history

#   else:
#     img = image_to_base64(image)
#     system = SystemMessage(content=system_prompt)
#     human_prompt = HumanMessage(content=[
#         {"type": "text", "text": text},
#         {
#             "type": "image_url",
#             "image_url": f"data:image/jpeg.base64,{img}"
#         }
#     ])
#     formatted_prompt = system_ + human_prompt
#     response = vis_llm.invoke(formatted_prompt)
#     history += [(None, response.content)]
#     return history

# # Interface Code
# with gr.Blocks() as app:
#     with gr.Row():
#         image_box = gr.Image(type="filepath")

#         chatbot = gr.Chatbot(
#             scale = 2,
#             height=650,
#         )
#     text_box = gr.Textbox(
#             placeholder="Enter text and press enter, or upload an image",
#             container=False,
#         )

#     text_box.submit(query_message,
#                         [text_box,image_box,chatbot],
#                         chatbot
#                         ).then(chat_bot,
#                                 [text_box,image_box,chatbot],
#                                 chatbot
#                                 )
# app.queue()
# app.launch(debug=True)


<ipython-input-19-bf67a31f23a6>:79: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d6b75f3cda5a874a4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d6b75f3cda5a874a4f.gradio.live


In [ ]:
# text_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# vis_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-preview-image-generation")


# image = Image.open(path_to_image)

# system_prompt = SystemMessage(content=system_prompt)
# human_prompt = HumanMessage(content=[
#     {"type": "text", "text": input().str()},
#     {"type": "image", "image": image}
# ])

# formattted_prompt = system_prompt + human_prompt
# response = model.invoke(
# )

# response = client.models.generate_content(
#     model="gemini-2.0-flash-preview-image-generation",
#     contents=contents, # If image is included in input, add image to list of contents
#     config=types.GenerateContentConfig(
#       response_modalities=['TEXT', 'IMAGE']
#     )
# )

# for part in response.candidates[0].content.parts:
#   if part.text is not None:
#     print(part.text)
#   elif part.inline_data is not None:
#     image = Image.open(BytesIO((part.inline_data.data)))
#     image.save('gemini-native-image.png')
#     image.show()

In [ ]:
# image = Image.open(path_to_image)
# # Function to interact with the chatbot
# def chat_bot(user_input, conversation_history=[]):
#   messages = conversation_history.copy()
#   messages.append({"role": "user", "content": [user_input, image]})

#   # Format with system prompt
#   fromatted_prompt = system_prompt + "\n\n"

#   # Add conversation history
#   for msg in messages:
#     role = "Assistant" if msg['role'] == "'assistant" else 'user'
#     fromatted_prompt += f"{role}: {msg['content']}\n\n"

#   fromatted_prompt += "Assistant: "

#   # Get response from model
#   response = model.invoke(
#       fromatted_prompt,
#       generation_config=dict(response_modalities=['TEXT', 'IMAGE']),
#       )

#   # Add response to history
#   messages.append({"role": "assistant", "content": response.content})

#   return response.content, messages